# Loading data experiments

## Loading data from football-data.co.uk 

In [ ]:
import pandas as pd
import requests

url = "https://www.football-data.co.uk/mmz4281/2425/E0.csv"
response = requests.get(url)
response.raise_for_status()

In [2]:
from io import BytesIO

df = pd.read_csv(BytesIO(response.content))

In [3]:
df.head(5)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA
0,E0,16/08/2024,20:00,Man United,Fulham,1,0,H,0,0,...,1.86,2.07,1.83,2.11,1.88,2.11,1.82,2.05,1.90,2.08
1,E0,17/08/2024,12:30,Ipswich,Liverpool,0,2,A,0,0,...,2.05,1.88,2.04,1.90,2.20,2.00,1.99,1.88,2.04,1.93
2,E0,17/08/2024,15:00,Arsenal,Wolves,2,0,H,1,0,...,2.02,1.91,2.00,1.90,2.05,1.93,1.99,1.87,2.02,1.96
3,E0,17/08/2024,15:00,Everton,Brighton,0,3,A,0,1,...,1.87,2.06,1.86,2.07,1.92,2.10,1.83,2.04,1.88,2.11
4,E0,17/08/2024,15:00,Newcastle,Southampton,1,0,H,1,0,...,1.87,2.06,1.88,2.06,1.89,2.10,1.82,2.05,1.89,2.10


In [4]:
# discover categorical columns
df.select_dtypes(include=['object']).columns.tolist()

['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTR', 'HTR', 'Referee']

## Data cleaning

In [8]:
# convert columns to lower snake case
df.columns = df.columns.str.lower().str.replace(" ", "_").str.replace("-", "_")

# set date as datetime
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# set season
df['season'] = "2425"

In [11]:
import sys
sys.path.append("..")

from config import get_required_columns

df_required = df[get_required_columns()]

df_required.head(5)

,div,date,hometeam,awayteam,referee,fthg,ftag,ftr,hthg,htag,...,hf,af,hy,ay,hr,ar,whh,whd,wha,season
0,E0,2024-08-16,Man United,Fulham,R Jones,1,0,H,0,0,...,12,10,2,3,0,0,1.65,4.2,5.00,2425
1,E0,2024-08-17,Ipswich,Liverpool,T Robinson,0,2,A,0,0,...,9,18,3,1,0,0,8.50,5.5,1.33,2425
2,E0,2024-08-17,Arsenal,Wolves,J Gillett,2,0,H,1,0,...,17,14,2,2,0,0,1.18,7.0,17.00,2425
3,E0,2024-08-17,Everton,Brighton,S Hooper,0,3,A,0,1,...,8,8,1,1,1,0,2.60,3.5,2.70,2425
4,E0,2024-08-17,Newcastle,Southampton,C Pawson,1,0,H,1,0,...,15,16,2,4,1,0,1.35,5.5,8.00,2425


## Loading epl data to s3

In [13]:
from io import BytesIO

import boto3
from uuid import uuid4

client = boto3.client('s3')

bucket_name = f'test-{uuid4()}'

# Create a new S3 bucket
client.create_bucket(
  Bucket=bucket_name, 
  CreateBucketConfiguration={
    'LocationConstraint': client.meta.region_name
  }
)

# Upload the DataFrame to S3
file_name = '2425_E0.parquet'

parquet_buffer = BytesIO()
df.to_parquet(parquet_buffer, index=False)
client.put_object(Bucket=bucket_name, Body=parquet_buffer.getvalue(), Key=file_name)

{'ResponseMetadata': {'RequestId': 'HQMNFJQRG80CS752',
  'HostId': 'DLKsBgg0pvF+HYd0183Z4NdLgtMMqAKD2GB5oraJuhdfjTm4Rzkqo9tLgNSso3QrKjiqNICzqRI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'DLKsBgg0pvF+HYd0183Z4NdLgtMMqAKD2GB5oraJuhdfjTm4Rzkqo9tLgNSso3QrKjiqNICzqRI=',
   'x-amz-request-id': 'HQMNFJQRG80CS752',
   'date': 'Sun, 10 Aug 2025 20:36:38 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"0051246ec92c385f7772841b79c88ddd"',
   'x-amz-checksum-crc32': 'Lw0MCA==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"0051246ec92c385f7772841b79c88ddd"',
 'ChecksumCRC32': 'Lw0MCA==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

## Loading data to postgres

In [14]:
import sys
sys.path.append("..")

from config import get_config

config = get_config()

In [15]:
# Dump the configuration to JSON
config.model_dump_json()

'{"app_name":"epl-predictions","postgres_user":"admin","postgres_password":"admin","postgres_server":"localhost","postgres_port":5432,"postgres_db":"epl_predictions","table_name":"english_league_data","aws_region":"eu-south-1","aws_access_key_id":"**********","aws_secret_access_key":"**********","prefect_table_name_variable":"table-name","prefect_cloud_api_key":"**********"}'

In [16]:
from sqlalchemy import create_engine, text

engine = create_engine(config.database_url)

In [17]:
table_name = "test_table"

In [18]:
df.to_sql(
    name=table_name,
    con=engine,
    if_exists='replace',
    index=False,
    method='multi'
)

-1

## Clean up

In [19]:
client.delete_object(Bucket=bucket_name, Key=file_name)
client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': '20E2FSQ6P8APKAAC',
  'HostId': 'Xbp/d3DLMcLYy3fWU3Lep49vfgp9fiwrjsWCzI+CudEc/6HPAmy8ZGEa3obY5bmFtYuAhvo6kCA=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'Xbp/d3DLMcLYy3fWU3Lep49vfgp9fiwrjsWCzI+CudEc/6HPAmy8ZGEa3obY5bmFtYuAhvo6kCA=',
   'x-amz-request-id': '20E2FSQ6P8APKAAC',
   'date': 'Sun, 10 Aug 2025 20:44:15 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [20]:
with engine.connect() as connection:
    connection.execute(text(f"DROP TABLE {table_name}"))
    connection.commit()  # Explicitly commit the transaction